In [1]:
import requests
import json
import os
from dotenv import load_dotenv

api_key = os.getenv("TVFY_API_KEY")

In [2]:
def fetch_additional_info(member_id, api_key):
    url = f"https://theyvoteforyou.org.au/api/v1/people/{member_id}.json?key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [4]:
result = fetch_additional_info(10748, api_key)
print(result)

{'id': 10748, 'latest_member': {'id': 615, 'name': {'first': 'Alan', 'last': 'Tudge'}, 'electorate': 'Aston', 'house': 'representatives', 'party': 'Liberal Party'}, 'rebellions': 0, 'votes_attended': 2063, 'votes_possible': 2273, 'offices': [], 'policy_comparisons': [{'policy': {'id': 5, 'name': 'government administered paid parental leave', 'description': 'the federal government should administer the paid parental leave scheme rather than employers', 'provisional': False, 'last_edited_at': '2014-09-19T13:09:30+10:00'}, 'agreement': '100', 'voted': True}, {'policy': {'id': 12, 'name': 'voluntary student union fees', 'description': 'university students should be able to choose whether or not to pay student services and amenities fees (also called student union fees)', 'provisional': False, 'last_edited_at': '2017-10-27T13:23:41+11:00'}, 'agreement': '100', 'voted': True}, {'policy': {'id': 13, 'name': 'increasing or removing the Government debt limit', 'description': "the federal govern

In [5]:
# Mapping dictionary for 'Effective Party' column.
effective_party_map = {
    'Country Liberal Party': 'Liberal National Party',
    'DPRES': 'Australian Labor Party',
    'Liberal Party': 'Liberal National Party',
    'National Party': 'Liberal National Party',
    'PRES': 'Australian Labor Party',
    'Centre Alliance': 'Independent',
    'CWM': 'Liberal National Party',
    'Katter\'s Australian Party': 'Independent',
    'SPK': 'Australian Labor Party'
}


def flatten_member_info(member):
    latest_member = member['latest_member']
    party = latest_member['party']
    effective_party = effective_party_map.get(party, party)  # Get effective party from map, or use original party if not found

    additional_info = member.get('additional_info', {})

    return {
        'id': member['id'],
        'name': f"{latest_member['name']['first']} {latest_member['name']['last']}",
        'electorate': latest_member['electorate'],
        'house': latest_member['house'],
        'party': latest_member['party'],
        'effective_party': effective_party,
        'rebellions': additional_info.get('rebellions'),
        'votes_attended': additional_info.get('votes_attended'),
        'votes_possible': additional_info.get('votes_possible'),
        'offices': [office['position'] for office in additional_info.get('offices', [])]
    }


In [6]:
flattened_member = flatten_member_info(result)
print(flattened_member)

{'id': 10748, 'name': 'Alan Tudge', 'electorate': 'Aston', 'house': 'representatives', 'party': 'Liberal Party', 'effective_party': 'Liberal National Party', 'rebellions': None, 'votes_attended': None, 'votes_possible': None, 'offices': []}


In [7]:
def load_members_from_files(filename):
    # Check if the file with all members exists
    if os.path.exists(filename):
        # Load data from the file if it exists
        with open(filename, 'r') as f:
            return json.load(f)
    else:
        print("No local member data found.")
        return None

In [9]:
# OS setup
house_filename = "../data/parliament/house.json"
current_house = load_members_from_files(house_filename)



In [10]:
print(len(current_house))

151


In [11]:
current_house.append(flattened_member)

In [12]:
print(len(current_house))

152


In [13]:
with open(house_filename, 'w') as f:
    json.dump(current_house, f)